In [3]:
!pip install torch torchvision
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hnj04wr_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-hnj04wr_
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
import clip
import torch
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)


In [ ]:
image = preprocess(Image.open("/")).unsqueeze(0).to(device)


In [ ]:
import clip
import torch
from PIL import Image
import os
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
print("Model ViT-L/14 loaded successfully.")
image_path = "/"

if not os.path.exists(image_path):
    print(f"خطا: فایل تصویر در مسیر '{image_path}' یافت نشد.")
    print("لطفاً تصویر خود را آپلود کرده و مسیر آن را به درستی وارد کنید.")
else:
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    print("Image loaded and preprocessed successfully.")
    text_descriptions = [
        "A photo of a brown dog playing with a blue ball in the grass", 
        "A golden retriever chasing a ball outdoors", 
        "A pet enjoying a sunny day", 
        "There is a dog and a ball in the image", 
        "A cat sleeping on a red sofa", 
        "A landscape view of mountains at sunset" 
    ]

    text_inputs = clip.tokenize(text_descriptions).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity_probs[0].topk(3)

    print("\n--- نتایج تطابق تصویر با توصیف‌ها ---")
    print("مدل، تصویر شما را با توصیف‌های زیر مقایسه کرده و این نتایج را برگردانده است:\n")

    for i, (value, index) in enumerate(zip(values, indices)):
        print(f"#{i+1}: \"{text_descriptions[index]}\"")
        print(f"   - احتمال تطابق: {value.item() * 100:.2f}%")
        print("-" * 20)


100%|███████████████████████████████████████| 890M/890M [00:57<00:00, 16.2MiB/s]


Model ViT-L/14 loaded successfully.
Image loaded and preprocessed successfully.

--- نتایج تطابق تصویر با توصیف‌ها ---
مدل، تصویر شما را با توصیف‌های زیر مقایسه کرده و این نتایج را برگردانده است:

#1: "A photo of a brown dog playing with a blue ball in the grass"
   - احتمال تطابق: 71.69%
--------------------
#2: "A golden retriever chasing a ball outdoors"
   - احتمال تطابق: 27.24%
--------------------
#3: "There is a dog and a ball in the image"
   - احتمال تطابق: 0.95%
--------------------
